In [2]:
import os
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, approx_count_distinct, avg, max, min, row_number
from pyspark.sql.window import Window

# Environment variables for Azure Data Lake Storage
storage_account_name = os.getenv("STORAGE_ACCOUNT_NAME")
storage_container_name = os.getenv("STORAGE_CONTAINER_NAME")
tf_var_client_id = os.getenv("TF_VAR_CLIENT_ID")
tf_var_client_secret = os.getenv("TF_VAR_CLIENT_SECRET")
tf_var_tenant_id = os.getenv("TF_VAR_TENANT_ID")

if not all(
    [
        storage_account_name,
        storage_container_name,
        tf_var_client_id,
        tf_var_client_secret,
        tf_var_tenant_id,
    ]
):
    raise ValueError("One or more environment variables are missing")

# Configure Spark session
spark = (
    SparkSession.builder.appName("IncrementalProcessing")
    .config("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
spark.conf.set(
    f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net",
    "OAuth",
)
spark.conf.set(
    f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
)
spark.conf.set(
    f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net",
    tf_var_client_id,
)
spark.conf.set(
    f"fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net",
    tf_var_client_secret,
)
spark.conf.set(
    f"fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net",
    f"https://login.microsoftonline.com/{tf_var_tenant_id}/oauth2/token",
)

# Load Parquet files from ADLS
incremental_hotel_weather_path = f"abfss://{storage_container_name}@{storage_account_name}.dfs.core.windows.net/incremental-hotel-weather"
df = spark.read.format("parquet").parquet(incremental_hotel_weather_path)

# Show the loaded DataFrame
df.show(truncate=False)

ValueError: One or more environment variables are missing

In [ ]:
df_stream = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .schema(df.schema)
    .load(incremental_hotel_weather_path)
)

result_df = df_stream.groupBy("city", "wthr_date").agg(
    F.approx_count_distinct("id").alias("distinct_hotels"),
    F.avg("avg_tmpr_c").alias("avg_temperature"),
    F.max("avg_tmpr_c").alias("max_temperature"),
    F.min("avg_tmpr_c").alias("min_temperature"),
)

result_df.explain(extended=True)

== Parsed Logical Plan ==
'Aggregate ['city, 'wthr_date], ['city, 'wthr_date, approx_count_distinct('id, 0.05, 0, 0) AS distinct_hotels#467, avg('avg_tmpr_c) AS avg_temperature#469, max('avg_tmpr_c) AS max_temperature#471, min('avg_tmpr_c) AS min_temperature#473]
+- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@1a77144e,cloudFiles,List(),Some(StructType(StructField(address,StringType,true),StructField(avg_tmpr_c,DoubleType,true),StructField(avg_tmpr_f,DoubleType,true),StructField(city,StringType,true),StructField(country,StringType,true),StructField(geoHash,StringType,true),StructField(id,StringType,true),StructField(latitude,DoubleType,true),StructField(longitude,DoubleType,true),StructField(name,StringType,true),StructField(wthr_date,StringType,true),StructField(processed_date,TimestampType,true),StructField(year,IntegerType,true),StructField(month,IntegerType,true),StructField(day,IntegerType,true))),List(),None,Map(cloudFiles.format -> parquet, path -> abfss://data

In [ ]:
query = (
    result_df.writeStream.outputMode("complete")
    .format("memory")
    .queryName("result_df_query")
    .start()
)

In [ ]:
# Window specification to rank cities by distinct hotels and weather date
window_spec = Window.partitionBy("city").orderBy(col("distinct_hotels").desc(), col("wthr_date").desc())

# Add a row number to each partition
result_df_with_row_num = result_df.withColumn("row_number", row_number().over(window_spec))

# Filter to get the top 1 row per city
unique_cities_df = result_df_with_row_num.filter(col("row_number") == 1).drop("row_number")

# Get the top 10 cities
window_spec_top_10 = Window.orderBy(col("distinct_hotels").desc())
top_10_cities_df = unique_cities_df.withColumn("rank", row_number().over(window_spec_top_10)).filter(col("rank") <= 10).drop("rank")

# Show the resulting DataFrame
top_10_cities_df.show(truncate=False)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-682172774778850>:15
     12 top_10_cities_df = unique_cities_df.withColumn("rank", row_number().over(window_spec_top_10)).filter(col("rank") <= 10).drop("rank")
     14 # Show the resulting DataFrame
---> 15 top_10_cities_df.show(truncate=False)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:933, in DataFrame.show(self, n, truncate, vertical)
    924 except ValueError:
    925     raise PySparkTypeError(
    926         error_class="NOT_A_BOOLEAN",